This notebook shows that the precision of the methyLImp is higher than the precision of the cytosine mean in the cytosines, where methylation has high correlation with age.

In [1]:
import numpy as np
import pandas as pd
import utils

In [2]:
header = ["chromosome", "start_offset", "end_offset", "region_name", "num_of_changed_cyt", "NA", "average_diff"]
header = header + [f"garbage{i}" for i in range(20)]
dmrs = pd.read_csv("../data/dmrs_filtered_ncyto_ge3_abs_avgdiff_ge0.025.hg19.bed", header=0, names=header, sep='\t', index_col=False)
columns = list(dmrs.columns)
dmrs = dmrs[columns[0:4] + [columns[6]]]
dmrs

,chromosome,start_offset,end_offset,region_name,average_diff
0,chr1,967968,968065,dmr4,-0.052931
1,chr1,1240049,1240295,dmr7,-0.070996
2,chr1,1240328,1240370,dmr8,-0.086937
3,chr1,1368525,1368558,dmr10,-0.198932
4,chr1,1369134,1369159,dmr11,-0.098029
...,...,...,...,...,...
1154,chrX,150345780,150345841,dmr2587,0.053336
1155,chrX,150345962,150346032,dmr2588,0.026197
1156,chrX,152939401,152939453,dmr2591,0.030437
1157,chrX,153046343,153046440,dmr2592,0.094189


In [3]:
dmrs.sort_values("average_diff")

,chromosome,start_offset,end_offset,region_name,average_diff
828,chr3,184790538,184790549,dmr1839,-0.593909
983,chr7,2646739,2646806,dmr2206,-0.255013
862,chr4,55015808,55015822,dmr1914,-0.243383
694,chr2,204649340,204649375,dmr1513,-0.240711
860,chr4,55015508,55015743,dmr1912,-0.221609
...,...,...,...,...,...
810,chr3,51741027,51741522,dmr1794,0.220112
1013,chr7,101961701,101962183,dmr2266,0.224659
933,chr6,2953225,2953276,dmr2093,0.233165
952,chr6,36247907,36247918,dmr2133,0.234788


In [4]:
data  = pd.read_csv("../data/filtered_cytosines_freq_preprocessed.tsv", sep='\t')
data

,chromosome,position,OD10,OD11,OD12,OD13,OD14,OD15,OD16,OD17,...,YD20,YD21,YD2,YD3,YD4,YD5,YD6,YD7,YD8,YD9
0,chr1,10497,71.956,93.640,94.410,92.163,86.408,92.143,91.304,69.029,...,92.880,80.769,85.714,73.761,84.277,80.408,94.667,92.623,83.992,89.620
1,chr1,10525,93.704,90.813,95.679,89.375,93.528,95.714,93.103,89.737,...,90.000,87.179,91.081,91.329,92.332,85.537,92.857,89.617,92.073,94.684
2,chr1,10542,100.000,100.000,100.000,100.000,100.000,100.000,100.000,100.000,...,100.000,100.000,100.000,100.000,100.000,100.000,99.647,99.708,99.781,100.000
3,chr1,10589,100.000,NaN,NaN,0.000,100.000,NaN,100.000,NaN,...,NaN,85.714,92.308,NaN,100.000,50.000,0.000,91.379,91.892,81.818
4,chr1,10609,95.455,NaN,NaN,0.000,100.000,NaN,63.636,NaN,...,NaN,85.714,94.737,NaN,100.000,100.000,NaN,96.296,97.222,80.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808443,chrY,59021089,60.563,75.000,90.000,61.429,92.754,56.190,56.604,100.000,...,69.565,65.000,66.038,64.368,72.000,49.275,48.750,80.769,83.333,80.645
2808444,chrY,59026010,85.714,80.247,84.615,80.117,90.909,84.946,81.818,91.045,...,87.234,81.522,77.193,83.065,87.059,77.143,82.308,88.889,77.838,76.923
2808445,chrY,59033031,78.431,91.827,90.196,89.286,84.000,80.000,89.744,72.581,...,90.385,79.592,91.667,94.118,66.667,100.000,95.000,94.737,86.047,85.246
2808446,chrY,59033041,79.167,97.101,86.275,85.185,92.593,92.000,84.615,82.258,...,87.821,83.838,85.714,87.500,88.235,85.714,95.000,73.684,85.366,90.000


In [5]:
chr = "chr3"
start_pos = 184790538
end_pos = 184790549

chromosome_coords = utils.get_chromosome_coords(data["chromosome"])
i, j = chromosome_coords[chr]
example_data = data.loc[i:j].reset_index().drop("index", axis=1)
ind1, ind2 = np.where((example_data["position"] > start_pos) & (example_data["position"] < end_pos))[0]

In [27]:
from functools import partial

rmse_lst = []
mae_lst = []

n, m = example_data.shape

nans_mask = np.zeros((n, m-2), dtype=bool)
nans_mask[ind1:ind2+1] = np.random.binomial(1, 0.1, size=(2, m-2))
nans_mask = np.concatenate((np.zeros((n, 2), dtype=bool), nans_mask), axis=1)
print(nans_mask.sum())
data_with_nans = example_data.copy()
data_with_nans[nans_mask] = np.nan

rows = np.any(nans_mask, axis=1)

def impute_nbp(data_with_nans_):
    return utils.impute_1000bp(data_with_nans_, eps=300, impute_positions=nans_mask[:, 2:]).values[:, 2:]
methyLimp = partial(utils.new_methyLImp, eps=3000, rows=rows)

for impute in [methyLimp, impute_nbp]:
    imputed_data = impute(data_with_nans)
    
    diffs = example_data.values[nans_mask].astype(np.float64) - imputed_data[nans_mask[:, 2:]].astype(np.float64)
    rmse = np.sqrt(np.nanmean((diffs ** 2)))
    mae = np.nanmean(np.abs(diffs))

    rmse_lst.append(rmse)
    mae_lst.append(mae)

diffs_with_cytosine_means = (np.nanmean(data_with_nans.values[:, 2:].astype(float), axis=1)[:, None] - example_data.values[:, 2:].astype(float))[nans_mask[:, 2:]]

rmse = np.sqrt(np.nanmean(diffs_with_cytosine_means ** 2))
mae = np.nanmean(np.abs(diffs_with_cytosine_means))

rmse_lst.append(rmse)
mae_lst.append(mae)

diffs_with_people_means = (np.nanmean(data_with_nans.values[:, 2:].astype(float), axis=0)[None, :] - example_data.values[:, 2:].astype(float))[nans_mask[:, 2:]]

rmse = np.sqrt(np.nanmean(diffs_with_people_means ** 2))
mae = np.nanmean(np.abs(diffs_with_people_means))

rmse_lst.append(rmse)
mae_lst.append(mae)

10


100%|██████████| 2/2 [00:00<00:00, 24.33it/s]


number of used methylations: 80
14 of them imputed by methyLImp


100%|██████████| 10/10 [00:00<00:00, 2542.62it/s]


In [28]:
rmse_lst

[26.129038489057766, 12.57254356524566, 25.641583537913174, 53.79354560414478]

In [29]:
mae_lst

[14.395073264626182, 8.518350000000003, 21.988580882352938, 51.9154282234521]

In [50]:
rmse_lst = []
mae_lst = []
n_nans_lst = []

n, m = example_data.shape

n_runs = 10
for _ in range(n_runs):
    nans_mask = np.zeros((n, m-2), dtype=bool)
    nans_mask[ind1:ind2+1] = np.random.binomial(1, 0.05, size=(2, m-2))
    nans_mask = np.concatenate((np.zeros((n, 2), dtype=bool), nans_mask), axis=1)
    n_nans_lst.append(nans_mask.sum())
    data_with_nans = example_data.copy()
    data_with_nans[nans_mask] = np.nan

    rows = np.any(nans_mask, axis=1)

    def impute_nbp(data_with_nans_):
        return utils.impute_1000bp(data_with_nans_, eps=300, impute_positions=nans_mask[:, 2:]).values[:, 2:]
    methyLimp = partial(utils.new_methyLImp, eps=3000, rows=rows)

    for impute in [methyLimp, impute_nbp]:
        imputed_data = impute(data_with_nans)
        
        diffs = example_data.values[nans_mask].astype(np.float64) - imputed_data[nans_mask[:, 2:]].astype(np.float64)
        rmse = np.sqrt(np.nanmean((diffs ** 2)))
        mae = np.nanmean(np.abs(diffs))

        rmse_lst.append(rmse)
        mae_lst.append(mae)

    diffs_with_cytosine_means = (np.nanmean(data_with_nans.values[:, 2:].astype(float), axis=1)[:, None] - example_data.values[:, 2:].astype(float))[nans_mask[:, 2:]]

    rmse = np.sqrt(np.nanmean(diffs_with_cytosine_means ** 2))
    mae = np.nanmean(np.abs(diffs_with_cytosine_means))

    rmse_lst.append(rmse)
    mae_lst.append(mae)

    diffs_with_people_means = (np.nanmean(data_with_nans.values[:, 2:].astype(float), axis=0)[None, :] - example_data.values[:, 2:].astype(float))[nans_mask[:, 2:]]

    rmse = np.sqrt(np.nanmean(diffs_with_people_means ** 2))
    mae = np.nanmean(np.abs(diffs_with_people_means))

    rmse_lst.append(rmse)
    mae_lst.append(mae)

100%|██████████| 2/2 [00:00<00:00, 35.60it/s]


number of used methylations: 80
8 of them imputed by methyLImp


100%|██████████| 2/2 [00:00<00:00, 63.62it/s]


number of used methylations: 40
3 of them imputed by methyLImp


100%|██████████| 2/2 [00:00<00:00, 42.07it/s]


...

100%|██████████| 2/2 [00:00<00:00, 69.23it/s]


number of used methylations: 40
8 of them imputed by methyLImp


100%|██████████| 2/2 [00:00<00:00, 63.30it/s]


number of used methylations: 40
6 of them imputed by methyLImp


100%|██████████| 4/4 [00:00<00:00, 4106.02it/s]


In [51]:
methylimp_rmse, nbp_rmse, cytosine_rmse, people_rmse = np.array(rmse_lst).reshape(n_runs, 4).T

In [52]:
benchmark_table_rmse = pd.DataFrame({"methyLImp": methylimp_rmse,
                                    "nbp": nbp_rmse,
                                    "Cytosine mean": cytosine_rmse,
                                    "People mean": people_rmse,
                                    "n_nans": n_nans_lst})
benchmark_table_rmse

,methyLImp,nbp,Cytosine mean,People mean,n_nans
0,53.342864,32.012845,44.539532,47.186575,4
1,13.821840,7.629250,5.323162,46.326954,1
2,43.041755,23.295819,32.659366,46.823192,7
3,50.151737,26.285370,40.555350,32.169230,5
4,67.128757,37.623493,49.622053,19.892583,5
5,35.355277,8.688442,26.777315,45.514108,2
6,41.335693,18.262618,31.029536,43.934298,6
7,0.003352,6.510500,22.440622,63.344092,2
8,43.311960,21.738841,33.016977,45.804696,6
9,59.094865,28.542630,45.293097,43.364371,4


In [53]:
methylimp_mae, nbp_mae, cytosine_mae, people_mae = np.array(mae_lst).reshape(n_runs, 4).T

In [54]:
benchmark_table_mae = pd.DataFrame({"methyLImp": methylimp_mae,
                                    "nbp": nbp_mae,
                                    "Cytosine mean": cytosine_mae,
                                    "People mean": people_mae,
                                    "n_nans": n_nans_lst})
benchmark_table_mae

,methyLImp,nbp,Cytosine mean,People mean,n_nans
0,35.454142,21.327938,37.226917,42.855940,4
1,13.821840,7.629250,5.323162,46.326954,1
2,28.788733,17.535714,27.784576,41.302796,7
3,43.485938,21.433750,32.579136,29.775264,5
4,63.807632,33.805800,45.479586,19.005601,5
5,25.000164,7.863000,26.684622,37.933375,2
6,28.160494,14.572150,25.284555,39.816066,6
7,0.003352,6.510500,22.440622,63.344092,2
8,29.576054,17.061000,30.303000,38.960193,6
9,44.371786,23.365812,38.820250,38.885961,4
